In [1]:
from IPython.display import clear_output
!pip install twarc #Twarc
!pip install tweepy # Tweepy 3.8.0
!pip install argparse #Argparse 3.2
!pip install xtract #Xtract 0.1 a3
!pip install wget #Wget 3.2
clear_output()

In [2]:
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd

dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-10-05/2021-10-05_clean-dataset.tsv.gz?raw=true"


#Downloads the dataset (compressed in a GZ format)
#!wget dataset_URL -O clean-dataset.tsv.gz
wget.download(dataset_URL, out='clean-dataset.tsv.gz')

'clean-dataset.tsv.gz'

In [3]:
#Unzips the dataset and gets the TSV dataset
with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
    with open('clean-dataset.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

#Deletes the compressed GZ file
os.unlink("clean-dataset.tsv.gz")

#Gets all possible languages from the dataset
df = pd.read_csv('clean-dataset.tsv',sep="\t")
print(len(df))
lang_list = df.lang.unique()
lang_list= sorted(np.append(lang_list,'all'))
lang_picker = widgets.Dropdown(options=lang_list, value="all")
lang_picker

176673


Dropdown(options=('all', 'am', 'ar', 'bg', 'bn', 'ca', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'eu', '…

In [4]:
#Creates a new clean dataset with the specified language (if specified)
filtered_language = lang_picker.value

#If no language specified, it will get all records from the dataset
if filtered_language == "":
  copyfile('clean-dataset.tsv', 'clean-dataset-filtered.tsv')

#If language specified, it will create another tsv file with the filtered records
else:
  filtered_tw = list()
  current_line = 1
  with open("clean-dataset.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")

    if current_line == 1:
      filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))

      for line in tsvreader:
        if line[3] == filtered_language:
          filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))
        current_line += 1

  print('\033[1mShowing first 5 tweets from the filtered dataset\033[0m')
  print(filtered_tw[1:(6 if len(filtered_tw) > 6 else len(filtered_tw))])

  with open('clean-dataset-filtered.tsv', 'w') as f_output:
      for item in filtered_tw:
          f_output.write(item)

Showing first 5 tweets from the filtered dataset
['1445237563321692162\t2021-10-05\t04:01:00\ten\tNULL\n', '1445237566949797891\t2021-10-05\t04:01:01\ten\tNULL\n', '1445237569437020162\t2021-10-05\t04:01:02\ten\tNULL\n', '1445237574507712516\t2021-10-05\t04:01:03\ten\tNULL\n', '1445237578584760325\t2021-10-05\t04:01:04\ten\tNULL\n']


In [5]:
import json
import tweepy
from tweepy import OAuthHandler

# Authenticate
CONSUMER_KEY = "~" #@param {type:"string"}
CONSUMER_SECRET_KEY = "~" #@param {type:"string"}
ACCESS_TOKEN_KEY = "~-~" #@param {type:"string"}
ACCESS_TOKEN_SECRET_KEY = "~" #@param {type:"string"}

#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)


In [6]:
from IPython.display import clear_output
import sys

!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_acquisition/get_metadata.py -O get_metadata.py

clear_output()

In [7]:
!python get_metadata.py -i clean-dataset-filtered.tsv -o hydrated_tweets -k api_keys.json

#clear_output()

Your twitter api credentials are valid.
hydrated_tweets
tab seperated file, using \t delimiter
total ids: 99033
metadata collection complete
creating master json file
currently getting 0 - 100
currently getting 100 - 200
currently getting 200 - 300
currently getting 300 - 400
currently getting 400 - 500
currently getting 500 - 600
currently getting 600 - 700
currently getting 700 - 800
currently getting 800 - 900
Rate limit reached. Sleeping for: 127
currently getting 900 - 1000
currently getting 1000 - 1100
currently getting 1100 - 1200
currently getting 1200 - 1300
currently getting 1300 - 1400
currently getting 1400 - 1500
currently getting 1500 - 1600
currently getting 1600 - 1700
currently getting 1700 - 1800
currently getting 1800 - 1900
currently getting 1900 - 2000
currently getting 2000 - 2100
currently getting 2100 - 2200
currently getting 2200 - 2300
currently getting 2300 - 2400
currently getting 2400 - 2500
currently getting 2500 - 2600
currently getting 2600 - 2700
curren

In [ ]:
from IPython.display import clear_output
import nltk

nltk.download('stopwords')
nltk.download("punkt")

!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/parse_json_lite.py -O parse_json_lite.py
!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/fields.py -O fields.py

!pip install emot --upgrade
!pip install emoji --upgrade
!pip install autocorrect

clear_output()

In [ ]:
!python parse_json_lite.py hydrated_tweets_short.json p

#clear_output()

parse_json_lite.py:89: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  tweet_df = pd.io.json.json_normalize(data)


In [ ]:
df = pd.read_csv('hydrated_tweets_short.tsv', sep="\t")
df.head()

result = Counter(" ".join(df['text'].values.tolist()).split(" ")).items()

ParserError: ignored

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

no_top_unique_words = "20"

df = pd.read_csv('hydrated_tweets_short.tsv', sep="\t")

result = Counter(" ".join(df['text'].values.tolist()).split(" ")).items()
df2 = pd.DataFrame(result)
df2.columns =['Word', 'Frequency']
df2 = df2[df2.Word != ""] #Deletes the empty spaces counted
df2 = df2.sort_values(['Frequency'], ascending=[False]) #Sort dataframe by frequency (Descending)

print('\033[1mTop '+no_top_unique_words+' most unique words used from the dataset\033[0m \n')
print(df2.head(int(no_top_unique_words)).to_string(index=False)) #Prints the top N unique words used
print("\n")
df3 = df2.head(int(no_top_unique_words))
df3.plot(y='Frequency', kind='pie', labels=df3['Word'], figsize=(9, 9), autopct='%1.1f%%', title='Top '+no_top_unique_words+' most unique words used from the data')


ParserError: ignored

--------------------------------------------

In [ ]:
## sample로 1000개만 추출하는 코드! 굳이!!!
import random 

no_samples = "1000" #@param {type:"string"}
list_tweets = None

with open("hydrated_tweets_short.json", "r") as myfile:
    list_tweets = list(myfile)

if int(no_samples) > len(list_tweets):
    no_samples = len(list_tweets)

sample = random.sample(list_tweets, int(no_samples))

file = open("sample_data.json", "w")
for i in sample:
  file.write(i)
file.close() #This close() is important

In [ ]:
# Filtering first
import pandas as pd
import numpy as np
import json
import sys
import string
import re
# This will load the fields list
import fields
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO
import emoji
from nltk.corpus import stopwords
from collections import Counter

fieldsFilter = fields.fields

fileN = sys.argv[1]
preprocess = sys.argv[2]

data = []

with open(fileN, 'r') as f:
    for line in f:
        data.append(json.loads(line))

def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = re.sub(u'('+re.escape(emot)+')', "_".join(EMOTICONS_EMO[emot].replace(",","").split()), text)
        return text

def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(re.escape(emot), "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
        return text

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

def remove_urls(text):
    result = re.sub(r"http\S+", "", text)
    return(result)

def remove_twitter_urls(text):
    clean = re.sub(r"pic.twitter\S+", "",text)
    return(clean)

def remove_mentions(text):
    mentions = re.compile(r"@\w+ ?")
    return mentions.sub(r"", text)

def remove_others(text) :
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

punc_list = string.punctuation  # you can self define list of punctuation to remove here
def remove_punctuations(text):
    translator = str.maketrans("", "", punc_list)
    return text.translate(translator)

def remove_whitespaces(text):
    return " ".join(text.split())

STOPWORDS = set(stopwords.words('english'))
def stop_words(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

cnt = Counter()
freq = set([w for (w, wc) in cnt.most_common(10)])
def freq_words(text):
    return " ".join([word for word in str(text).split() if word not in freq])



tweet_df = pd.io.json.json_normalize(data)
# Cleaner solution in case some of the fields in the list are non existent and/or have typos
tweet_df = tweet_df.loc[:, tweet_df.columns.isin(fieldsFilter)]

# lowercasing
tweet_df['text'] = tweet_df['text'].str.lower()

# convert emoji and emoticons
tweet_df['text'] = tweet_df['text'].apply(lambda x : convert_emoticons(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : convert_emojis(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : give_emoji_free_text(x))

# remove 
tweet_df['text'] = tweet_df['text'].apply(lambda x : remove_urls(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : remove_twitter_urls(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : remove_mentions(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : remove_punctuations(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : remove_whitespaces(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : remove_others(x))

# removal stop and frequent words
tweet_df['text'] = tweet_df['text'].apply(lambda x : stop_words(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x : freq_words(x))

with open(fileN[:-5]+".tsv",'w') as write_tsv:
    write_tsv.write(tweet_df.to_csv(sep='\t', index=False))